In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


# 토크나이저 관련 경고 무시하기 위하여 설정
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# # device 지정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

사용 디바이스: cuda


In [3]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

voice_df = pd.read_csv('/content/drive/MyDrive/AI_Hackathon/data/voice_df_n.csv', sep='\t')


In [4]:
voice_df.isnull().sum()

Unnamed: 0     0
content       50
target         0
dtype: int64

In [5]:
voice_df = voice_df.dropna()

In [6]:
# !pip install -U imbalanced-learn

In [7]:
from sklearn.utils import resample
import pandas as pd

# voice_df 데이터프레임에서 content와 target 컬럼을 추출
X = voice_df['content']
y = voice_df['target']

# 클래스 0와 클래스 1을 분리
class_0 = voice_df[voice_df['target'] == 0]
class_1 = voice_df[voice_df['target'] == 1]

# 클래스 0을 10배 복제하여 오버샘플링
oversampled_class_1 = resample(class_1, replace=True, n_samples=len(class_0))

# 클래스 0와 클래스 1을 합쳐 오버샘플링된 데이터 생성
voice = pd.concat([class_0, oversampled_class_1])


In [8]:
voice.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [9]:
voice.isnull().sum()

content    0
target     0
dtype: int64

In [10]:
voice.sum()

content    친애하다 판사 라는 관련 드라마무한도전 완전 폐지 돼다 없어지다 무한도전 자주 자다...
target                                                  1711
dtype: object

In [11]:
df_shuffled = sklearn.utils.shuffle(voice)

train, test = train_test_split(voice, test_size=0.2)

In [12]:
train

,content,target
799,이루어지다 보다 돼다 기다리다 어차피 입증 받다 그렇다 천만 본인 주머니 수표 가방...,1
1721,진운 웃기다 크다 진짜,0
1768,그렇다 마는 잖다,0
172,피해자 따로 따로 경찰서 방문 없이 고소장 사이 오다 보고 기범 보내다 드리다 피해...,1
1707,맞다 특히 특히 흐름 끊기다 아예 드라마 보다 근데 흐름 끊기다 보다 진짜 열심히 보다,0
...,...,...
324,피해자 개인 정보 침해 고객 센터 신고 센터 기범 클릭 화면 넘어가다 나다 화면 넘...,1
1502,앙금 너무 사적 영역 이라서 민망하다,0
483,고객 피해자 기범 자리 빼다 원만 납부 이자 이자 내주다 피해자 기범 해주다 에서도...,1
706,고객 피해자 그렇다 기범 고객 만요 전산 나본 확인 연락 드리다 피해자 기범 기범 ...,1


In [13]:
CHECKPOINT_NAME = 'kykim/bert-kor-base'

In [14]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 124.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.5 MB/s eta 0:00:00


In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


# KoBERT 모델 및 토크나이저 로드
model_name = "monologg/kobert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 이진 분류이므로 num_labels=2

# 모델의 파라미터와 버퍼를 지정한 장치로 이동
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [16]:
train['content'] = train['content'].astype(str)
test['content'] = test['content'].astype(str)

In [17]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=300):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['content']
        label = self.data.iloc[idx]['target']

         #텍스트를 토큰화
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_attention_mask=True, return_tensors='pt', return_token_type_ids=False)

        # 딕셔너리 형태로 데이터 구성
        input_dict = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze().to(device),
            'labels': torch.tensor(label, dtype=torch.long).to(device)
}

        return input_dict

# 데이터셋 및 DataLoader 생성
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
train_dataset = CustomDataset(train, tokenizer)
test_dataset = CustomDataset(test, tokenizer)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=False)



In [ ]:
num_epochs = 10
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU가 사용 가능한 경우 GPU 사용
# 옵티마이저 정의 및 초기화
optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)

    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            predicted_labels = torch.argmax(outputs.logits, dim=1)
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += len(labels)

    accuracy = total_correct / total_samples

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_train_loss:.4f}, Accuracy: {accuracy:.4f}")

# 학습된 모델을 저장
torch.save(model.state_dict(), "kobert_binary_classification_model.pth")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 0.1600, Accuracy: 1.0000
Epoch 2/10, Loss: 0.0184, Accuracy: 1.0000
Epoch 3/10, Loss: 0.0082, Accuracy: 1.0000
Epoch 4/10, Loss: 0.0048, Accuracy: 1.0000
Epoch 5/10, Loss: 0.0032, Accuracy: 1.0000
Epoch 6/10, Loss: 0.0023, Accuracy: 1.0000


# 오버샘플링 결과

* 클래스가 0인 데이터를 2배 오버샘플링->0의 데이터셋이 더 많지만 토큰수로 비교했을 때 1이 더 많음

  일반대화: 0(예측 성공)

  상담대화: 0(예측 성공)

  보이스피싱: 0(예측 실패)

  : 모두 0으로 예측하기 때문에 클래스 0인 데이터가 많아서 overfitting 의심됨.


* 클래스 1을 클래스 0의 개수만큼 오버 샘플링->클래스가 1인 샘플수가 적기 때문에

  일반대화: 0(예측 성공)

  상담대화: 0(예측 성공)

  보이스피싱: 0(예측 실패)

위 방법들은 모두 틀린 방법

1. 데이터셋 추가하기

2. 클래스가 1인 데이터셋 분할(max_length로 입력으로 사용하는 토큰 개수를 조정했기 때문에 토큰수에 따라서 샘플링을 할 필요가 없었음. 따라서, max_length를 너무 작게 설정하여 데이터셋이 너무 많이 잘리지 않도록 max_length를 더 크게 설정하고 클래스가 1인 데이터의 대화를 분할해보기)

  


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax

# KoBERT 모델 및 토크나이저 초기화
model_path = "kobert_binary_classification_model.pth"
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=2)
model.load_state_dict(torch.load(model_path))
model.eval()

# 입력 문자열
#input_text = " 안녕, 오랜만이야. 어떻게 지내? 안녕! 나도 오랜만이야. 잘 지내고 있어. 너는 어때? 나도 괜찮아. 일상생활이 바빠서 좀 피곤해. 이해해. 업무나 학교 어때? 무슨 일 하고 있어?" #일반 대화
#input_text = "안녕하세요, 어떻게 도와드릴까요? 안녕하세요, 최근에 업무 스트레스 때문에 정말 힘들어요. 네, 이해해요. 어떤 일이 스트레스를 유발하고 있나요? 업무 부하가 매우 많고, 기간 내에 모든 것을 해내기 어려워서 스트레스 받고 있어요. 그런 일들은 정말 어렵겠죠. 어떻게 스트레스 관리를 시도하고 계신가요? 아직은 특별한 방법을 시도하지는 않았어요. 어떻게 관리해야 할지 모르겠어요. 스트레스 관리에 도움이 될 수 있는 몇 가지 방법을 고려해 볼까요? 운동이나 명상을 시도해보는 것도 좋은 방법이 될 수 있어요. 그런 아이디어를 시도해 볼까요. 다른 사람들은 어떻게 스트레스를 관리하나요? 다른 사람들은 다양한 방법을 사용하고 있어요. 일단, 자신에게 가장 적합한 방법을 찾는 것이 중요해요. 또한, 가족이나 친구들과 이야기하고, 지원을 받는 것도 도움이 될 수 있어요." #일반대화(상담)
input_text = "여보세요 네 안녕하세요 서울중앙지방검찰청 00부 담당 형사 000입니다. 000님 맞으십니까? 맞는데요 다름이 아니고 혹시 김00님을 아십니까? 지금 김00님이 경남은행에서 000님 명의로 대포통장을 개설하였습니다. 예? 지금 피해사례가 220건이 넘고 대포통장 개설해준 명목으로 000님께서 명의를 도용당하신 것인지, 범죄에 가담한 것인지 확인하려고 전화드렸는데요. 네?? 대포통장이요? 저 그런 적 없는데요 원래는 사건 참고인 조사는 방문조사 하시라고 우편으로 내용을 보내드리는데 긴급한 사항이고 방문하기 어려운 상황이시다 하시면 전화로 조사가 이루어집니다. 그래서 이렇게 유선으로 참고인조사가 이루어지고 있고요. 아 예.. 네 그럼 일단 대포통장 범죄에 000님이 가담하셨는지 아니면 정말 피해자이신건지 몇가지 사항으로 확인드리겠습니다. 괜찮으십니까? 네 그래주세요. 그 전에 본인확인 할텐데요. 녹취가 되고요. 정확하게 진술해주십시요." #보이스피싱 대화

# 입력 문자열을 토큰화하고 모델에 입력하기
input_ids = tokenizer.encode(input_text, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors='pt')
with torch.no_grad():
    outputs = model(input_ids)
logits = outputs.logits

# 예측 클래스 및 확률 계산
predicted_class = torch.argmax(logits, dim=1).item()
class_probabilities = softmax(logits, dim=1)[0].tolist()

# 결과 출력
print(f"입력 문자열: {input_text}")
print(f"예측 클래스: {predicted_class}")
print(f"클래스 확률: {class_probabilities}")
